# Fine-Tuning Model

# Setup

In [ ]:
# Python version
import sys 
print(sys.version)

In [ ]:
# Environment Variables
from dotenv import load_dotenv
import yaml
import os

# Load env
load_dotenv()

In [ ]:
# Torch config
from torch import cuda, bfloat16, float16
import torch

# Torch options
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Parameters

In [ ]:
# Load parameters from YAML file
import os

# Change the current working directory to the directory containing the YAML file
os.chdir('C:/Users/polri/Desktop/Git_TFM/TFM_LAW_LLM')

# Load parameters from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
# Use optimum
use_optimum = config["use_optimum"]

# Show
use_optimum

# References

# Directory

In [ ]:
# Set directory to file location
from pathlib import Path
import sys
notebook_location = Path(os.path.abspath(''))
os.chdir(notebook_location)

# Get the current working directory
current_directory = os.getcwd()
current_directory

# Libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# Warnings
import warnings
warnings.filterwarnings("ignore")

# Device

In [ ]:
# Setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# CUDA information
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # Convert bytes to GB
    cached_memory = torch.cuda.memory_reserved(0) / (1024**3)  # Convert bytes to GB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert bytes to GB
    available_memory = total_memory - cached_memory
    print('Allocated:   ', round(allocated_memory, 1), 'GB')
    print('Cached:      ', round(cached_memory, 1), 'GB')
    print('Available:  ', round(available_memory, 1), 'GB')
    print('Total:      ', round(total_memory, 1), 'GB')

# Clean memory
torch.cuda.empty_cache()
gc.collect()

# Pinecone

In [ ]:
# Init pinecone
pinecone = Pinecone(api_key = "03b29f67-c297-4462-825b-13ce23b3d577")

pc = Pinecone(api_key = pinecone)
# Connect
index_name = 'lawllm-unstructured-database'
index = pinecone.Index(index_name)

# Index stats
index.describe_index_stats()

# Embedding Model

In [ ]:
# Model ID
embed_model_id = config["embedding_model"]

# Show
embed_model_id

In [ ]:
# Embed model
embed_model = HuggingFaceEmbeddings(
    model_name = embed_model_id,
    model_kwargs = {'device': device},
    encode_kwargs = {'device': device, 'batch_size': 32}
) 

# Show
embed_model

In [ ]:
# CUDA information
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # Convert bytes to GB
    cached_memory = torch.cuda.memory_reserved(0) / (1024**3)  # Convert bytes to GB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert bytes to GB
    available_memory = total_memory - cached_memory
    print('Allocated:   ', round(allocated_memory, 1), 'GB')
    print('Cached:      ', round(cached_memory, 1), 'GB')
    print('Available:  ', round(available_memory, 1), 'GB')
    print('Total:      ', round(total_memory, 1), 'GB')

# Load LLM model

# Model ID
use_quantization = config["use_quantization"]

# Show
use_quantization

In [ ]:
# Select model
model_id = config["model"]

# Show
model_id

In [ ]:
# Tokenizer
import transformers
from transformers import AutoTokenizer

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

In [ ]:
# Set BNB configuration if quantization is enabled
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
) if use_quantization else None

In [ ]:
# Set model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code = True,
    quantization_config = bnb_config,
    device_map = "auto"
)

In [ ]:
# CUDA information
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # Convert bytes to GB
    cached_memory = torch.cuda.memory_reserved(0) / (1024**3)  # Convert bytes to GB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert bytes to GB
    available_memory = total_memory - cached_memory
    print('Allocated:   ', round(allocated_memory, 1), 'GB')
    print('Cached:      ', round(cached_memory, 1), 'GB')
    print('Available:  ', round(available_memory, 1), 'GB')
    print('Total:      ', round(total_memory, 1), 'GB')

# Command

In [ ]:
# Get pre-prompt
pre_prompt = config["pre_prompt"]

# Create prompt context
prompt_context = config["prompt_context"]

In [ ]:
# General template
general_template = pre_prompt + prompt_context + "A continuación se proporciona el contexto: {context}" + " " + "pregunta: {query}"

In [ ]:
# Mistral template
mistral_template = "<s>[INST]" + pre_prompt + prompt_context +  "A continuación se proporciona el contexto: [/INST] {context}" + "</s>" + "[INST] pregunta: {query} [/INST]"

In [ ]:
# Google template
google_template = f"""
<start_of_turn>user
{pre_prompt}. {prompt_context} A continuación se proporciona el contexto: 
Contexto: {{context}} 
Pregunta: {{query}}
<end_of_turn>
<start_of_turn>model
Respuesta: """

In [ ]:
# Define the final template selection logic
if "mistral" in model_id:
    template = mistral_template
    selected_template_message = "Mistral template selected."
elif "google" in model_id:
    template = google_template
    selected_template_message = "Google template selected."
else:
    template = general_template
    selected_template_message = "Default template selected."

# Print out the selected template message
print(selected_template_message)

In [ ]:
# Prompt Template
prompt = PromptTemplate(
    template = template, 
    input_variables = ["context", "query"]
)

prompt

# LLM Pipeline

In [ ]:
# Define pipeline with parameters from config file
generate_text = transformers.pipeline(
    model = model,
    tokenizer = tokenizer,
    task = 'text-generation',
    return_full_text = config["return_full_text"],
    max_new_tokens = config["max_new_tokens"],
    repetition_penalty = config["repetition_penalty"],
    temperature = config["temperature"],
    pad_token_id = tokenizer.eos_token_id,
    batch_size = 1
)

# HF pipeline
llm = HuggingFacePipeline(pipeline = generate_text)

# Create llm chain 
llm_chain = LLMChain(llm = llm, prompt = prompt)

# Vector Store

In [ ]:
# Text field
text_field = "text"  

# Vector store
vectorstore = PineconeVectorStore(index, embed_model, text_field)  

# Show
vectorstore

# Test models

In [ ]:
# Simple context
context = "Eres una API con conocimientos legales. Debes responder a preguntas en Español. Si no conoces la respuesta, admítelo."

# Query
query = 'Explícame el Artículo 245 del Código Penal de España referente a ocupaciones ilegales de bienes inmuebles'

# Find closer Documents

In [ ]:
# Similarity output
similarity_output = vectorstore.similarity_search_with_score(query, k = config['top_k_docs'])

# Context preprocessed
context_processed = [{"context": doc.page_content, "score": score} for doc, score in similarity_output]

# Show
context_processed[0:3]

# Re-Ranking

In [ ]:
# Model ID
reranking_model = config["reranking_model"]

# Show
reranking_model 

In [ ]:
# Model ID
top_reranked_docs = config["top_reranked_docs"]

# Show
top_reranked_docs

In [ ]:
# Extracting 'title' keys
final_context = [entry['context'] for entry in context_processed]

In [ ]:
# Cross encoder
cross_encoder = CrossEncoder(reranking_model)

# Show
cross_encoder

In [ ]:
# CUDA information
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    allocated_memory = torch.cuda.memory_allocated(0) / (1024**3)  # Convert bytes to GB
    cached_memory = torch.cuda.memory_reserved(0) / (1024**3)  # Convert bytes to GB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # Convert bytes to GB
    available_memory = total_memory - cached_memory
    print('Allocated:   ', round(allocated_memory, 1), 'GB')
    print('Cached:      ', round(cached_memory, 1), 'GB')
    print('Available:  ', round(available_memory, 1), 'GB')
    print('Total:      ', round(total_memory, 1), 'GB')

In [ ]:
# Re-ranking
text_field = 'context'
ranked_context = rank_documents(cross_encoder, text_field, query, context_processed)

In [ ]:
# Sort
sorted_ranked_context = dict(sorted(ranked_context.items())[:top_reranked_docs])

In [ ]:
# Format
sorted_ranked_context = list(sorted_ranked_context.values())

In [ ]:
# Show
sorted_ranked_context[0:3]

# Others

In [ ]:
# 1. Prepare your Dataset
# Define your dataset and data loaders here

# 2. Fine-Tuning Configuration
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# 3. Load Pre-Trained Model
model_name = "bert-base-uncased"  # Example pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4. Define Fine-Tuning Loop
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset            # evaluation dataset
)

# Train the model
trainer.train()

# 5. Evaluation
trainer.evaluate()

# 6. Save Fine-Tuned Model
model.save_pretrained("./fine_tuned_model")